In [1]:
import nltk
import string
import csv
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import math

In [2]:
#bagian b: method tokenisasi
#method ini menggunakan method split untuk memisahkan setiap dokummen menjadi term berdasarkan spasi
def token(d):
    k=d.split()
    return k

#bagian c : method hilangkan
#method ini digunakan untuk menghilangkan tanda baca dan angka
#method ini menggunakan source code replace yang terdapat pada library numpy. tanda baca dan angka dimasukkan ke dalam variabel 
#symbols. dokumen yang mengandung karakter yang terdapat pada symbol akan di replace dengan spasi
def hilangkan(d):
    symbols = "12345678910!\'"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        d= np.char.replace(d, i, ' ')
        hasil=np.array2string(d)
    return hasil

#bagian d: method casefolding
#method ini digunakan untuk merubah term yang terdapat huruf kapital menjadi huruf kecil semua dengan menggunakan method lower 
def casefolding(d):
    hasil=d.lower()
    return hasil

#bagian e: cleaning
#project ini tidak menggunakan step cleaning karena data yang digunakan berupa data artikel ilmiah, dimana data artikel ilmiah 
#tidak terdapat tag, link maupun script

#bagian f: filtering
#pada bagian filtering terdapat method stopword yang digunakan untuk menghapus kata stoplist. daftar stoplist diambil dari kamus tala
#yang di download dari internet, lalu dimasukkan ke dalam file stop_words.txt. Alasan menggunakan stoplist dari tala karena
#kamus tala efektif digunakan untuk data berupa artikel ilmiah. Method remove digunakan untuk menghapus term dari dokumen jika term
#tersebut terdaftar pada stopword
def stopword(document):
    file=open('stop_words.txt',"rt")
    stopword=file.read()
    doc=token(document)
    stop=token(stopword)
    for d in doc:
        for s in stop:
            if d==s:
                doc.remove(d)
    return doc

#bagian g: stemming
#method stemming menggunakan library sastrawi untuk mempermudah proses stemming.
def stemming(dokumen):
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    hasil=[]
    for a in range(len(dokumen)):
        hasil.append(stemmer.stem(dokumen[a]))
    return hasil

In [3]:
#method preprocessing digunakan untuk menyatukan method method dari bagian b-g. data yang akan diproses pada method ini harus 
#diproses parsing terlebih dahulu
def preprocessing(data):
    #hilangkan tanda
    hasil1=hilangkan(data)
    #mengecilkan huruf
    hasil2=casefolding(hasil1)
    #menghilangkan kata tidak penting(filtering dengan stopword)
    hasil3=stopword(hasil2)
    #melakukan stemming
    hasil4=stemming(hasil3)
    return hasil4

In [4]:
#membuka file data berformat csv
df=pd.read_csv("dataartikelilmiah.csv")
#penambahan source code dibawah karena terdapat penambahan label artikel pada file csv
dataCsv=df['artikel']
#mengubah ke array dan disimpan pd variabel arr
arr=dataCsv.values

#bagian a: parsing
#bagian ini digunakan untuk mengubah 1 file data artikel menjadi beberapa artikel. hasil dari proses parsing akan menjadi 30 dokumen
#karena di dalam file artikel terdapat 30 data artikel

def parsing(arr):
    #digunakan untuk mengetahui jumlah data berdasarkan file text yang dibuat 
    jumlahData=0
    for dt in range(len(arr)):
        #perubahan source code dibawah untuk mencegah data ke 30 dapat diinput ke dalam file baru, karena sebelumnya hanya berisi file kosong
        #k=arr[dt]
        #data=np.array2string(k, precision=2, suppress_small=True)
        dataBaru=open(str(dt)+".txt","w+")
        teks=dataBaru.write(arr[dt])
        teks=open(str(dt)+".txt","rt")
        doc=teks.read()
        jumlahData+=1
    hasil=[]
    for a in range(len(arr)):
        #perubahan dibawah untuk load semua data, karena sebelumnya data yg terakhir hanya berisi array kosong
        #teks=open(str(a)+".txt","rt")
        doc=arr[a]
        #setelah parsing maka setiap dokumen akan masuk ke method preprocessing
        hasil.append(preprocessing(doc))
    return [hasil,jumlahData]

#digunakan untuk variabel yang menampung hasil akhir dari semua tahapan preprocessing
hasilAkhir= parsing(arr)
jumlahd= hasilAkhir[1]
hasilPreprocessing = hasilAkhir[0]

In [5]:
#method untuk menghitung jumlahterm tiap dokumen atau raw
def nilaiRawTerm(kumpulanTerm,hasilPrepos):
    rawData=[]
    #source code dibawah digunakan untuk menghitung jumlah term yang terdapat pada tiap dokumen
    for term in kumpulanTerm:
        rawTermDat=[]
        for i in range(len(hasilPrepos)):
            rawTerm = 0
            for j in range(len(hasilPrepos[i])):
                #term akan terus bertambah sesuai dengan jumlah term yang terdapat pada tiap dokumen
                if term == hasilPrepos[i][j]:
                    rawTerm += 1
            rawTermDat.append(rawTerm)
        rawData.append(rawTermDat)
    return rawData

#method untuk menghitung pembobotan TF
def hitungTF(data):
    dataTF = [sublist[:] for sublist in data]
    for i in range (len(dataTF)):
        for j in range (len(dataTF[i])):
            #perhitungan bobot term frequency. term frequency berupa hasil dari method "nilaiRawTerm" yang berisi jumlah term yang terkandung pada tiap dokumen
            if dataTF[i][j]>1:
                #jika jumlah term tiap dokumen lebih dari satu maka dihitung dengan rumus dibawah. jika jumlah termnya nol maka nilainya tetap
                dataTF[i][j]= 1 +math.log10(dataTF[i][j])
    return dataTF

#method untuk menghitung frequency dokumen
def hitungDokFrekuensi(dataTF):
    dokFreq=[]
    for i in range(len(dataTF)):
        freq=0
        for j in range(len(dataTF[i])):
            #source code dibawah digunakan untuk menghitung frekuensi dokumen berdasarkan TF
            if dataTF[i][j]>0:
                freq+=1
        dokFreq.append(freq)
    return dokFreq

#method untuk menghitung nilai idf
def hitungIDF(dataTF1,jumlahDok):
    #mengambil hasil frekuensi dokumen dari method hitungDokFrekuensi
    frekDokumen = hitungDokFrekuensi(dataTF1)
    hasilIdf=[]
    #perhitungan IDF berdasarkan dokumen frekuensi
    for i in range(len(frekDokumen)):
        hasilIdf.append(math.log10(jumlahDok/frekDokumen[i]))
    return hasilIdf

#method untuk menghitung nilai tfidf
def hitungTFIDF(dataTF,dataIDF):
    dataTfIdf = [sublist[:] for sublist in dataTF]
    for i in range(len(dataTF)):
        #perhitungan tf idf tiap raw
        for j in range(len(dataTF[i])):
            dataTfIdf[i][j] = dataTF[i][j]*dataIDF[i]
    return dataTfIdf

In [6]:
#digunakan untuk menggabungkan semua term yang dihasilkan dari hasil preprocessing
kumpulanTerm = []
for a in range(len(hasilPreprocessing)):
    for b in hasilPreprocessing[a]: 
        #source code dibawah digunakan untuk mencegah penulisan term yang sama
        if b not in kumpulanTerm: 
            kumpulanTerm.append(b) 
kumpulanTerm

['teknologi',
 'mixed',
 'reality',
 'buat',
 'interaksi',
 'user',
 'aplikasi',
 'nyata',
 'rupa',
 'padu',
 'dunia',
 'maya',
 'virtual',
 'world',
 'real',
 'harap',
 'guna',
 'dapat',
 'paham',
 'informasi',
 'beri',
 'jadwal',
 'salah',
 'proses',
 'laku',
 'tiap',
 'organisasi',
 'capai',
 'tuju',
 'masalah',
 'jadi',
 'bagai',
 'sumber',
 'daya',
 'yang',
 'besar',
 'usaha',
 'perintah',
 'institusi',
 'didik',
 'untuk',
 'penuh',
 'butuh',
 'masyarakat',
 'sedia',
 'kembang',
 'nama',
 'push',
 'notification',
 'mungkin',
 'perangkat',
 'client',
 'notifikasi',
 'kirim',
 'server',
 'otomatis',
 'mcdonald s',
 'gerak',
 'bidang',
 'jual',
 'makan',
 'cepat',
 'saji',
 'indonesia',
 'milik',
 'cabang',
 'outlet',
 'sebar',
 'daerah',
 '',
 'internet',
 'ini',
 'pesat',
 'desa',
 'pencil',
 'minim',
 'imbang',
 'lahan',
 'timbang',
 'lombok',
 'punya',
 'potensi',
 'tanda',
 'tingkat',
 'unjung',
 'pulau',
 'tahun',
 'akses',
 'wisatawan',
 'asing',
 'mudah',
 'telah',
 'bangun',

In [7]:
#menghitung nilai term tiap baris
rawTerm = nilaiRawTerm(kumpulanTerm,hasilPreprocessing)
#menghitung nilai term frekuensi
hasilTF = hitungTF(rawTerm)
#menghitung nilai IDF
nilaiIdf = hitungIDF(hasilTF,jumlahd)
#menghitung nilai TF-IDF
nilaiTfIdf = hitungTFIDF(hasilTF,nilaiIdf)
#hasil TF-IDF digabungkan berdasarkan term dan dokumennya dengan bentuk dictionary
hasilAkhirrTFIDF = dict()
for a in range(len(kumpulanTerm)):
    hasilAkhirrTFIDF.update({kumpulanTerm[a]:nilaiTfIdf[a]})
hasilAkhirrTFIDF

{'teknologi': [0.7468318977627837,
  0.0,
  0.8479137864342865,
  0.0,
  0.5740312677277188,
  0.0,
  0.0,
  0.0,
  0.9196325277978485,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5740312677277188,
  0.0,
  0.0,
  0.0,
  0.8479137864342865,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5740312677277188,
  0.5740312677277188],
 'mixed': [1.4771212547196624,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'reality': [1.1760912590556813,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.1760912590556813,
  0.0],
 'buat': [1.1760912590556813,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.1760912590556813,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0

In [8]:
#method untuk menyimpan data hasil perhitungan TF-IDF kedalam format plain text (.csv) 
def outputDataTfIdf(data, term):
    with open('outputDataTfIdf.csv', 'w', newline='') as csvfile:
        fileTFIDF = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        #membuat header data pada file csv
        fileTFIDF.writerow(term)
        dataTFIDF=list(map(list, zip(*data)))
        #menulis file csv dengan data 
        for a in range(len(dataTFIDF)):
            fileTFIDF.writerow(dataTFIDF[a])
        
    return fileTFIDF

In [9]:
#memanggil method untuk menyimpan data hasil perhitungan TF-IDF kedalam format plain text (.csv)
output=outputDataTfIdf(nilaiTfIdf, kumpulanTerm)

In [10]:
#clustering
import random

#hitung jarak
def hitungJarak(data,pusat):
    hasil=[]
    for c in range(len(data)):
        j=[]
        for i in range(len(data[c])):
            x= data[c][i]
            y = pusat[i]
            jarak1 = (y-x)**2
            j.append(jarak1)
        a = np.sum(j)
        b = math.sqrt(a)
        hasil.append(b)
    return hasil

#menghitung rata-rata centroid
def rataCentroid(centroid):
    centroid11 = list(map(list, zip(*centroid)))
    hasil=[]
    for a in range(len(centroid11)):
        c=0
        n=0
        for b in range(len(centroid11[a])):
            c=c+centroid11[a][b]
            n= n+1
        rata= c/n
        hasil.append(rata)
    return hasil

def kMeans(nilaiTfIdf,jumlahd):
    dataClustering = list(map(list, zip(*nilaiTfIdf)))
    #menentukan centroid awal dalam bentuk data ke-
    centroid=[]
    for a in range(3):
        b = random.randint(0, jumlahd)
        centroid.append(b)
    #penataan centroid yang telah didapatkan dari proses diatas
    pusat=[]
    for a in centroid:
        pusat.append(dataClustering[a])
    #mulai perulangan
    iterasi=0
    aa= 0
    cc= 0
    centro= pusat
    
    clusterLama={}    
    while iterasi <20:
        eucle=[]
        aa=aa+1
        #perhitungan jarak eucledean
        for b in range(len(centro)):
            jarak = hitungJarak(dataClustering,centro[b])
            eucle.append(jarak)
        
        #mencari jarak minimal dan penentuan cluster
        jarakMinimal = list(map(list, zip(*eucle)))
        cluster={}
        for a in range(len(jarakMinimal)):
            #bernilai 20 karena mencari kemungkinan bobot yg paling besar tapi tidak sampai 20
            maks=40
            #mencari jarak minimal
            for b in range(len(jarakMinimal[a])):
                if jarakMinimal[a][b] < maks:
                    maks = jarakMinimal[a][b]
                    cluster.update({a:b})
        
        #update centroid
        cluster1 = list(cluster.values())
        centroid1=[]
        centroid2=[]
        centroid3=[]
        for a in range(len(dataClustering)):
            if (cluster1[a] == 0):
                centroid1.append(dataClustering[a])
            elif (cluster1[a] == 1):
                centroid2.append(dataClustering[a])
            elif (cluster1[a] == 2):
                centroid3.append(dataClustering[a])
        centroidUpdate=[]       
        for a in range(3):
            if (a == 0):
                cc = rataCentroid(centroid1)
            elif (a== 1):
                cc = rataCentroid(centroid2)
            elif (a== 2):
                cc= rataCentroid(centroid3)
            centroidUpdate.append(cc)
        
        if cluster == clusterLama:
            break
        else:
            clusterLama=cluster
            centro = centroidUpdate
        
        
    return (centroidUpdate,cluster,clusterLama,aa)

In [11]:
hasil = kMeans(nilaiTfIdf,jumlahd)
hasilIterasi=hasil[3]
hasilCluster=hasil[2]
centroidAkhir=hasil[0]

In [12]:
hasilCluster

{0: 2,
 1: 1,
 2: 2,
 3: 2,
 4: 2,
 5: 2,
 6: 1,
 7: 0,
 8: 2,
 9: 1,
 10: 2,
 11: 2,
 12: 2,
 13: 2,
 14: 1,
 15: 2,
 16: 2,
 17: 2,
 18: 2,
 19: 1,
 20: 2,
 21: 2,
 22: 2,
 23: 2,
 24: 2,
 25: 2,
 26: 2,
 27: 1,
 28: 2,
 29: 1}

In [13]:
#hitung jarak
def jarakCluster(data,pusat):
    hasil=[]
    for c in range(len(data)):
        j=[]
        for i in range(len(data[c])):
            x = data[c][i]
            y = pusat[i]
            jarak1 = (y-x)**2
            j.append(jarak1)
        a = np.sum(j)
        b = math.sqrt(a)
        hasil.append(b)
    return hasil

nilai = list(map(list, zip(*nilaiTfIdf)))
#silhouette koefisien
cluster= list(hasilCluster.values())
#hitung a
jarakEval= []
for a in range(len(nilai)):
    jarak = jarakCluster(nilai,nilai[a])
    jarakEval.append(jarak)

hasilA=[]
c=0
n=0
for i in range(len(jarakEval)):
    for b in range(len(jarakEval[i])):
        d=jarakEval[i][b]
        if cluster[i]==cluster[b]:
            c=c+d
            n=n+1
    rata=c/n
    hasilA.append(rata)
hasilA

[8.144632151836799,
 7.906976066594497,
 7.858358261395262,
 7.792862417909512,
 7.538088011915415,
 7.688519983143493,
 7.679226063124429,
 7.617792254619434,
 7.584589743775949,
 7.58547693159311,
 7.694653662186005,
 8.207641016167921,
 8.167203705185484,
 8.185184746693443,
 8.167020679322011,
 8.140280944738134,
 8.114937668814717,
 8.093189510437337,
 8.095843507982266,
 8.085744553245268,
 8.052100465524182,
 8.138909413342597,
 8.085477975168667,
 8.069442838646514,
 8.073504805336828,
 8.04222940440596,
 8.031704883769743,
 8.037383906909952,
 8.018929927129477,
 8.012187239840008]

In [14]:
#hitung b
hasilB=[]
c=0
n=0
for i in range(len(jarakEval)):
    for a in range(len(jarakEval[i])):
        d=jarakEval[i][a]
        if d!=0 and cluster[i]!=cluster[a]:
            c=c+d
            n=n+1
    rata=c/n
    hasilB.append(rata)
hasilB

[9.001041421024455,
 8.312678729774115,
 8.379632988722058,
 8.384364962764879,
 8.247893680592625,
 8.355497587747903,
 8.441736786114221,
 8.794471149023735,
 8.749337585991096,
 8.78473278996814,
 8.809763414104907,
 9.022970918595112,
 9.001913648566607,
 9.007419499438676,
 8.961495201375117,
 8.947367072957627,
 8.932153322625117,
 8.918057440533119,
 8.91888663116985,
 8.901464094504213,
 8.885124116474673,
 8.929967540779671,
 8.90243965781281,
 8.893111632494044,
 8.89357231925705,
 8.873449435331713,
 8.86637536530534,
 8.936368415497862,
 8.924161585349315,
 8.89676117652333]

In [15]:
hasilSilhouette=[]
for i in range(len(hasilA)):
    a = hasilA[i]
    b = hasilB[i]
    evaluasi = (b-a)/(max(a,b))
    hasilSilhouette.append(evaluasi)
hasilSilhouette

[0.0951455758427322,
 0.04880528604172846,
 0.06220734583822072,
 0.07054828212777486,
 0.08605902260202378,
 0.07982500115641633,
 0.090326285018036,
 0.13379757286882718,
 0.13312411719946327,
 0.13651591767760302,
 0.1265765832182906,
 0.09036157932714903,
 0.09272583319148316,
 0.09128416332740727,
 0.08865423728968753,
 0.09020375733312845,
 0.09149144940676239,
 0.09249412616997797,
 0.09228092666983796,
 0.09163880599850671,
 0.09375486937834779,
 0.08858465877111205,
 0.09176829206892459,
 0.09261873997375364,
 0.09220901168640055,
 0.09367496112797531,
 0.09413886138880519,
 0.10059841613387936,
 0.10143604523095434,
 0.09942651254003826]

In [16]:
rataRata=0
b=0
for a in range(len(hasilSilhouette)):
    rataRata=rataRata+hasilSilhouette[a]
    b=b+1
hasilAkhirSilhouette=rataRata/b
hasilAkhirSilhouette

0.09440920788684161

In [17]:
hasilIterasi

2